In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Parkinson

In [ ]:
import os
import pandas as pd
import numpy as np
import random, re

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)

seed_everything()

In [ ]:
HEAD = ['LFHD', 'RFHD', 'LBHD', 'RBHD']  # 머리 앞뒤
CENTER = ['C7', 'CLAV', 'STRN', 'T10']   # 7번경추, 쇄골, 명치, 10번흉추
WING = ['RBAK']  # 날개뼈

SHOULDER = ['LSHO', 'RSHO'] # 어깨
UPARM = ['LUPA', 'RUPA']    # 위팔
ELBOW = ['LELB', 'RELB']    # 팔꿈치
FOREARM = ['LFRM', 'RFRM']  # 아래팔
WRIST = ['LWRA', 'RWRA', 'LWRB', 'RWRB'] # 손목
FINGER = ['LFIN', 'RFIN']   # 손가락

SINGLE = CENTER + WING  # 한쪽만 존재하는 부위
DOUBLE = HEAD + SHOULDER + UPARM + ELBOW + FOREARM + WRIST + FINGER  # 양쪽인 부위
UPPERBODY = SINGLE + DOUBLE  # 상체

print(f'Upper Body: {len(UPPERBODY)} = Single {len(SINGLE)} + Double {len(DOUBLE)// 2 }*2')

BODY = HEAD + CENTER + WING
ARMS = SHOULDER + UPARM + ELBOW + FOREARM + WRIST + FINGER

LEGS = ['LASI', 'RASI', 'LPSI', 'RPSI',  # 골반 앞뒤
        'LTHI', 'RTHI',  # 대퇴
        'LKNE', 'RKNE',  # 무릎
        'LTIB', 'RTIB',  # 정강이
        'LHEE', 'RHEE',  # 발뒤꿈치
        'LANK', 'RANK',  # 복사뼈
        'LTOE', 'RTOE',  # 발가락
        ]

ALL = BODY + ARMS + LEGS
print(f'BODY {len(BODY)} + ARMS {len(ARMS)} + LEGS {len(LEGS)} = ALL {len(ALL)}')

### 피실험자 데이터 랜덤으로 불러오기

In [ ]:
#===== Config =====#
ptype = 'PD'
n_sample = 1
walk_direction = 'FW'

In [ ]:
df_info = pd.read_csv("csv_info.csv")
df_info['Category'].value_counts()

In [ ]:
def dataSampling(df, ptype, n_sample, walk_direction):
    sampled_df = df.loc[df['Category']==ptype].sample(n_sample)
    
    sampled_paths = []  # 랜덤추출한 참여자별 FW/BW 데이터 하나씩 선택
    for id in sampled_df.index:
        paths = list(map(lambda s: s.strip(), re.sub("'", "", sampled_df.loc[id, 'path'+walk_direction][1:-1]).split(',')))
        sampled_paths.extend(random.sample(paths, 1))
    
    return sampled_paths, list(sampled_df['Participant'])

In [ ]:
sampled_paths, _ = dataSampling(df_info, ptype, n_sample, walk_direction)
sampled_paths

In [ ]:
name, walk_order = sampled_paths[0][:-4].split('/')[-1].split('_')
print(f'[Participant INFO] Name: {name}, Type: {ptype}, Trial: {walk_order}')

df = pd.read_csv(sampled_paths[0])
df.rename(columns={"Unnamed: 0": "Frame"}, inplace=True)
df.head(10)

### x,y축 영점 조절

- x축 영점 = 양쪽 발뒤꿈치의 중점 (왼발은 -, 오른발은 +)
- y축 영점 = 양쪽 발뒤꿈치 중 뒷쪽에 위치한 발 (모든 y값이 항상 +)
- z축 영점 = 그대로 바닥을 0으로 한다.

In [ ]:
def shift_origin(df):
    df_adj = df.copy()

    heels = [ p + '_' + axis for p in ['LHEE','RHEE'] for axis in ['X','Y','Z'] ]
    heel_na = df[heels].isna().sum().sum()
    
    if (heel_na % 6 == 0) and (heel_na > 0):
        heel_na = df[heels].isna().sum() // 6
        print(f"[LHEE,RHEE] Starting Frame: {heel_na}")

    initials = np.round(df.loc[heel_na, heels].values, 2)
    print(f"Left HEEL Initial values: X = {np.round(initials[0],2)}, Y = {np.round(initials[1],2)}, Z = {np.round(initials[2],2)}")
    print(f"Right HEEL Initial values: X = {np.round(initials[3],2)}, Y = {np.round(initials[4],2)}, Z = {np.round(initials[5],2)}\n")

    new_origin = [(initials[0] + initials[3]) / 2, min(initials[[1,4]]), 0]
    print(f"New Origin: {np.round(new_origin, 2)}\n")

    p_xyz = [[p + '_X', p + '_Y', p + '_Z'] for p in ALL] # [[LFHD_X,Y,Z], [RFHD_X,Y,Z], ...]  컬럼 3개씩(x,y,z) 묶은 리스트
    for p in p_xyz:
        if df[p[0]].isna().sum() == 0:
            df_adj[p] = df[p] - new_origin
        else:
            n_nan = df[p[0]].isna().sum()
            df_adj.loc[n_nan:, p] = df.loc[n_nan:, p] - new_origin
            df_adj.loc[:n_nan, p[:2]] = 0
            df_adj.loc[:n_nan, p[2]] = df_adj.loc[n_nan, p[2]]

    return df_adj

    # df_adj.loc[num_na:, heels] = df.loc[num_na:, heels[:2]] - df.loc[num_na, heels[:-1]]

    # elif (num_na % 2 == 0) and (num_na > 0):
    #     print('LHEE, RHEE is unsymmetric w.r.t NaN')
        # left_na = df[markers[0]].isna().sum()
        # right_na = df[markers[3]].isna().sum()
        
        # left_initials = np.round(df.loc[left_na, heels[:3]].values, 2)
        # right_initials = np.round(df.loc[right_na, heels[3:]].values, 2)
        # print(f"Left HEEL Initial values: X = {left_initials[0]}, Y = {left_initials[1]}, Z = {left_initials[2]}")
        # print(f"Right HEEL Initial values: X = {right_initials[0]}, Y = {right_initials[1]}, Z = {right_initials[2]}\n")
    
    # if num_na > 0:
    #     print(f"[{p}] Starting Frame: {num_na}")
    #     df_adj.loc[:num_na, place_xyz[:-1]] = 0
    #     df_adj.loc[:num_na, place_xyz[-1]] = initials[2]

    # return df_adj

In [ ]:
# df_shifted = shift_origin(df)
shift_origin(df).head(7)

In [ ]:
df.head(7)